In [31]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, minmax_scale
from keras import layers
from keras.layers import Embedding, Dense, Merge, Dropout, Reshape, Flatten
from keras.models import Sequential
from keras.preprocessing.text import one_hot

In [32]:
df_user = pd.read_csv('users.csv')
df_business = pd.read_csv('business.csv')
df_review = pd.read_csv('train_reviews.csv')
# df_review.head()

In [33]:
df_user = df_user[['user_id', 'average_stars', 'fans', 'funny', 'name', 'review_count', 'useful']]
df_business = df_business[['business_id', 'address', 'attributes_Alcohol', 'attributes_BikeParking', 'attributes_BusinessAcceptsCreditCards', 'attributes_Caters', 'attributes_GoodForKids', 'attributes_HasTV', 'attributes_NoiseLevel', 'attributes_OutdoorSeating', 'latitude', 'longitude', 'review_count', 'stars']]
df_review = df_review[['user_id', 'business_id', 'stars']]

In [34]:
df_user_train = pd.merge(df_user, df_review, how='inner', on=['user_id'])[['user_id', 'average_stars', 'fans', 'funny', 'name', 'review_count', 'useful']]
df_business_train = pd.merge(df_business, df_review, how='inner', on=['business_id'])

In [39]:
df_business_train = df_business_train.drop(columns=['user_id', 'stars_y'])
df_user_train.set_index('user_id', inplace=True)
df_business_train.set_index('business_id', inplace=True)
df_user_train.head()

,average_stars,fans,funny,name,review_count,useful
user_id,,,,,,
UxfpKHGO2dfQCdS9xLLJow,2.83,0,2,James,6,7
UxfpKHGO2dfQCdS9xLLJow,2.83,0,2,James,6,7
UxfpKHGO2dfQCdS9xLLJow,2.83,0,2,James,6,7
sLrX2KGu3lc_JczAnsg0_Q,4.00,0,0,Brian,1,0
nmYitfmo-pQ1hJWDnTLwGg,3.33,0,0,D.,12,3


In [36]:
df_business_train.head()

,address,attributes_Alcohol,attributes_BikeParking,attributes_BusinessAcceptsCreditCards,attributes_Caters,attributes_GoodForKids,attributes_HasTV,attributes_NoiseLevel,attributes_OutdoorSeating,latitude,longitude,review_count,stars_x,user_id,stars_y
business_id,,,,,,,,,,,,,,,
KuxDPl6UYNLxFChPm0_MNw,595 Markham Street,full_bar,True,True,True,False,False,average,True,43.664125,-79.411886,146,4.0,UL0Cu3q46Zkxb2fAPdzbSA,3.0
KuxDPl6UYNLxFChPm0_MNw,595 Markham Street,full_bar,True,True,True,False,False,average,True,43.664125,-79.411886,146,4.0,R0kJv-G1rua8hNuE0iY6Xg,5.0
KuxDPl6UYNLxFChPm0_MNw,595 Markham Street,full_bar,True,True,True,False,False,average,True,43.664125,-79.411886,146,4.0,XUFfwI6YYF1oPgnqqCPCWw,3.0
KuxDPl6UYNLxFChPm0_MNw,595 Markham Street,full_bar,True,True,True,False,False,average,True,43.664125,-79.411886,146,4.0,qo8SplhjLRrIEo553JJnNA,5.0
KuxDPl6UYNLxFChPm0_MNw,595 Markham Street,full_bar,True,True,True,False,False,average,True,43.664125,-79.411886,146,4.0,cm3l-hlJLM87UwUG4xauQg,3.0


In [ ]:
le = LabelEncoder()
df_user_train[['name']] = df_user_train[['name']].astype(str)
df_user_train['name'] = le.fit_transform(df_user_train['name'].values)
df_business_train[['address', 'attributes_Alcohol', 'attributes_BikeParking', 'attributes_BusinessAcceptsCreditCards', \
             'attributes_Caters', 'attributes_GoodForKids', 'attributes_HasTV', 'attributes_NoiseLevel', \
             'attributes_OutdoorSeating']] \
= df_business_train[['address', 'attributes_Alcohol', 'attributes_BikeParking', 'attributes_BusinessAcceptsCreditCards', \
             'attributes_Caters', 'attributes_GoodForKids', 'attributes_HasTV', 'attributes_NoiseLevel', \
             'attributes_OutdoorSeating']].astype(str)
df_business_train[['address', 'attributes_Alcohol', 'attributes_BikeParking', 'attributes_BusinessAcceptsCreditCards', \
             'attributes_Caters', 'attributes_GoodForKids', 'attributes_HasTV', 'attributes_NoiseLevel', \
             'attributes_OutdoorSeating']]\
= df_business_train[['address', 'attributes_Alcohol', 'attributes_BikeParking', 'attributes_BusinessAcceptsCreditCards', \
             'attributes_Caters', 'attributes_GoodForKids', 'attributes_HasTV', 'attributes_NoiseLevel', \
             'attributes_OutdoorSeating']].apply(le.fit_transform)


In [ ]:
train_user = df_user_train.values
train_business = df_business_train.values

In [18]:
train_user = minmax_scale(train_user)
train_business = minmax_scale(train_business)


In [19]:
num_user = train_user.shape[0]
num_business = train_business.shape[0]

In [20]:
def my_model():
    k = 128
    model1 = Sequential()
    model1.add(Embedding(num_user + 1, k, input_length = 6))
    model1.add(Flatten())
    model2 = Sequential()
    model2.add(Embedding(num_business + 1, k, input_length = 13))
    model2.add(Flatten())

    model = Sequential()
    model.add(Merge([model1, model2], mode = 'concat'))
    model.add(Dropout(0.2))
    model.add(Dense(k, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(int(k/4), activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(int(k/16), activation = 'relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(1, activation = 'softmax'))
    return model

In [21]:
model = my_model()
model.compile(loss = 'mse', optimizer = "adam")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  # This is added back by InteractiveShellApp.init_path()


In [24]:
X_train = [train_user, train_business]
y_train = df_review['stars'].values

In [25]:
model.fit(X_train, y_train, batch_size = 100, epochs = 50)

Epoch 1/50
150232/150232 [==============================] - 770s 5ms/step - loss: 8.9651
Epoch 2/50
150232/150232 [==============================] - 786s 5ms/step - loss: 8.9651
Epoch 3/50
150232/150232 [==============================] - 780s 5ms/step - loss: 8.9651
Epoch 4/50
 19700/150232 [==>...........................] - ETA: 11:47 - loss: 8.9983

KeyboardInterrupt: 

In [41]:
df_user.set_index('user_id', inplace = True)
df_business.set_index('business_id', inplace=True)

In [42]:
queries = pd.read_csv('test_queries.csv')
queries.head()

,user_id,business_id
0,XEDaNNCTVAqPpvyX2zY03g,e880MGw6C6PJL-n3EJoY_g
1,E9WotdbCBpoeID-DHSQh1A,ExLXL_oQvI7oPB7TrVpGhg
2,8erAzkOo9KHs1Njo5bMm_Q,XKQsxvDMe9rAWO0P0Uwx-Q
3,eHJTLVy1bm7zSGVQlNH6KA,awI4hHMfa7H0Xf0-ChU5hg
4,KF2L4CDnmPz6diaH6TrxbA,jKrlc4xQIvfzDZ8eNfqxwg


In [54]:
df_user.loc['XEDaNNCTVAqPpvyX2zY03g']

average_stars    3.81
fans                2
funny               0
name              Dan
review_count       36
useful              0
Name: XEDaNNCTVAqPpvyX2zY03g, dtype: object